In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from corr_parc import (
    calculate_partial_correlation_df,
    apply_partial_correlation_criteria,
)
from merge import generate_merged_file

In [3]:
df_merged = generate_merged_file()

Quitamos `precipitacion_unidadmedida`


In [4]:
df_merged.drop("precipitacion_unidadmedida", axis=1, inplace=True)

In [5]:
df_merged.to_csv("processed_tables/merged_no_dams.csv", index=False)

### Correlación variables explicativas con variable dependiente con data estandarizada


In [6]:
scaler = StandardScaler()
exogen_data_df = df_merged.drop(["Date", "energy_price"], axis=1)
exogen_data = scaler.fit_transform(exogen_data_df)
exogen_data_df_standardized = pd.DataFrame(exogen_data, columns=exogen_data_df.columns)
scaled_df_merged = exogen_data_df_standardized.copy()
scaled_df_merged["energy_price"] = df_merged["energy_price"]

## Matriz de correlación


In [7]:
correlation_matrix = exogen_data_df.corr()
print(f"Correlation Matrix Original: {correlation_matrix.shape}")
correlation_matrix.style.background_gradient(cmap="coolwarm")

Correlation Matrix Original: (49, 49)


,brent_value,precipitacion_amazonas,precipitacion_arauca,precipitacion_atlantico,precipitacion_bogota,precipitacion_bolivar,precipitacion_caqueta,precipitacion_casanare,precipitacion_cesar,precipitacion_choco,precipitacion_guainia,precipitacion_guaviare,precipitacion_la guajira,precipitacion_magdalena,precipitacion_meta,precipitacion_narino,precipitacion_norte de santander,precipitacion_putumayo,precipitacion_quindio,precipitacion_risaralda,precipitacion_san andres providencia,precipitacion_sucre,precipitacion_vaupes,precipitacion_vichada,precipitacion_departamentos_represa,temp_AMAZONAS,temp_ARAUCA,temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA,temp_ATLANTICO,temp_BOGOTA,temp_BOLIVAR,temp_CAQUETA,temp_CASANARE,temp_CESAR,temp_CHOCO,temp_GUAINIA,temp_GUAVIARE,temp_LA GUAJIRA,temp_MAGDALENA,temp_META,temp_NARINO,temp_NORTE DE SANTANDER,temp_PUTUMAYO,temp_QUINDIO,temp_RISARALDA,temp_SUCRE,temp_VICHADA,temp_avg_represas,TRM
brent_value,1.000000,0.015180,0.034924,0.115467,0.252189,0.191135,0.051083,0.006283,0.204421,-0.312554,0.134976,0.183881,0.154511,0.358489,0.214289,0.186630,0.126432,-0.153138,0.052736,0.183884,0.134912,0.233963,0.001619,-0.013260,0.183865,-0.195409,-0.140329,-0.094225,-0.202322,-0.326451,-0.313565,-0.265061,-0.239194,-0.166155,-0.356461,-0.169176,-0.337520,0.040892,-0.125118,0.026172,0.085410,-0.295623,-0.011673,-0.356808,-0.192665,-0.304795,0.032238,-0.354904,-0.317626
precipitacion_amazonas,0.015180,1.000000,-0.066088,-0.075014,0.106463,-0.033369,-0.022714,-0.013511,-0.015286,0.083052,-0.059546,0.043675,-0.105463,-0.001357,-0.022306,0.036657,0.046852,-0.000844,0.066723,0.028251,-0.062482,0.008736,0.003200,0.008473,0.063766,-0.232514,0.073518,-0.060093,0.110331,0.052482,0.082042,0.055474,0.060518,0.086051,0.127459,0.101549,0.026618,0.017831,-0.011986,0.122759,0.101894,-0.008471,0.041728,-0.011253,0.026718,0.103934,-0.016458,0.036399,-0.011616
precipitacion_arauca,0.034924,-0.066088,1.000000,0.017710,0.045715,0.247388,0.209292,0.208583,0.061164,-0.017620,0.107836,0.117684,0.039089,0.109131,0.247661,-0.007031,-0.047415,0.056463,0.031036,0.096886,0.142209,0.190068,0.130980,0.287361,0.244383,0.038514,0.380051,0.185561,-0.041487,-0.123384,-0.046943,-0.208100,-0.149469,-0.063089,-0.109112,-0.071209,-0.242535,0.160982,0.006801,-0.131026,0.084866,0.017726,-0.094691,-0.038200,0.000818,0.021874,0.022685,-0.065212,-0.260864
precipitacion_atlantico,0.115467,-0.075014,0.017710,1.000000,0.013352,0.119168,0.019529,0.052842,0.036497,0.085676,0.116575,0.119205,0.230301,0.204079,0.060673,0.057876,0.053496,0.012250,0.022535,0.051031,0.137871,0.114213,-0.050822,-0.019426,0.163391,0.133536,-0.045853,0.117832,-0.168199,0.006932,-0.164992,0.070169,0.007875,-0.124734,-0.181007,0.068818,-0.023108,-0.049335,0.028422,0.089177,0.041105,-0.052018,0.086999,-0.092026,-0.067412,-0.184702,0.095447,-0.082236,-0.007299
precipitacion_bogota,0.252189,0.106463,0.045715,0.013352,1.000000,0.074195,0.104728,0.117971,0.109118,-0.002043,0.059264,0.215744,0.002575,0.242182,0.286513,0.285836,0.110124,0.033521,0.332314,0.235644,-0.014192,0.075289,0.072888,0.037687,0.434412,-0.056657,-0.061607,-0.010421,-0.036763,-0.116177,-0.068183,-0.121085,-0.107200,-0.085909,-0.045204,-0.034626,-0.205303,0.001194,-0.044992,-0.076165,0.032972,-0.155177,0.010893,-0.152004,-0.100928,-0.036242,-0.082595,-0.157813,-0.042113
precipitacion_bolivar,0.191135,-0.033369,0.247388,0.119168,0.074195,1.000000,0.063806,0.091918,0.142740,-0.004278,0.073121,0.167940,0.138250,0.235722,0.089413,0.090713,0.041052,-0.079292,0.048677,0.130465,0.120488,0.340527,0.145462,0.047074,0.252365,0.007794,0.158988,0.003807,-0.218628,-0.121354,-0.243621,-0.077080,-0.091759,-0.218361,-0.152417,-0.025532,-0.209518,-0.010243,-0.033582,-0.007320,0.043375,-0.110049,-0.008454,-0.201869,-0.137354,-0.191749,0.036608,-0.199072,-0.166742
precipitacion_caqueta,0.051083,-0.022714,0.209292,0.019529,0.104728,0.063806,1.000000,0.229139,0.085191,-0.036104,0.146477,0.223140,-0.008876,0.110767,0.417098,0

Se considera una alta correlación entre variables a valores superiores a 0.5 o menores a -0.5, esto puede causar multicolinealidad y afectar la eficiencia de la predicción de los modelos, por lo tanto, se van a analizar las variables que tienen esta alta correlación para realizar una selección de variables inicial.


In [8]:
partial_correlation_df = calculate_partial_correlation_df(scaled_df_merged)
partial_correlation_df.head()

,Variable,partial_correlation,abs_partial_correlation
0,brent_value,-0.198904,0.198904
1,precipitacion_amazonas,0.042761,0.042761
2,precipitacion_arauca,-0.030182,0.030182
3,precipitacion_atlantico,-0.038269,0.038269
4,precipitacion_bogota,0.096649,0.096649


In [9]:
columns_to_keep, _ = apply_partial_correlation_criteria(
    {
        "correlation_matrix": exogen_data_df_standardized.corr(),
        "partial_correlation_df": partial_correlation_df,
        "excluded": ["precipitacion_antioquia"],
    }
)
columns_to_keep

['precipitacion_caqueta',
 'precipitacion_cesar',
 'precipitacion_putumayo',
 'temp_AMAZONAS',
 'precipitacion_atlantico',
 'precipitacion_sucre',
 'precipitacion_risaralda',
 'precipitacion_arauca',
 'temp_CHOCO',
 'precipitacion_bolivar',
 'precipitacion_guainia',
 'precipitacion_narino',
 'temp_NORTE DE SANTANDER',
 'precipitacion_bogota',
 'precipitacion_la guajira',
 'temp_NARINO',
 'precipitacion_casanare',
 'precipitacion_san andres providencia',
 'TRM',
 'precipitacion_norte de santander',
 'precipitacion_amazonas',
 'precipitacion_magdalena',
 'temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
 'brent_value',
 'precipitacion_meta',
 'precipitacion_choco',
 'temp_ARAUCA',
 'temp_CAQUETA',
 'precipitacion_vaupes',
 'precipitacion_vichada',
 'precipitacion_guaviare']

In [10]:
df_without_correlation = exogen_data_df[columns_to_keep]
new_correlation_matrix = df_without_correlation.corr()
print(
    f"Correlation Matrix After deleting correlated variables: {new_correlation_matrix.shape}"
)
new_correlation_matrix.style.background_gradient(cmap="coolwarm")

Correlation Matrix After deleting correlated variables: (31, 31)


,precipitacion_caqueta,precipitacion_cesar,precipitacion_putumayo,temp_AMAZONAS,precipitacion_atlantico,precipitacion_sucre,precipitacion_risaralda,precipitacion_arauca,temp_CHOCO,precipitacion_bolivar,precipitacion_guainia,precipitacion_narino,temp_NORTE DE SANTANDER,precipitacion_bogota,precipitacion_la guajira,temp_NARINO,precipitacion_casanare,precipitacion_san andres providencia,TRM,precipitacion_norte de santander,precipitacion_amazonas,precipitacion_magdalena,temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA,brent_value,precipitacion_meta,precipitacion_choco,temp_ARAUCA,temp_CAQUETA,precipitacion_vaupes,precipitacion_vichada,precipitacion_guaviare
precipitacion_caqueta,1.000000,0.085191,0.299014,-0.023762,0.019529,0.066551,0.092406,0.209292,-0.103962,0.063806,0.146477,0.149357,0.004882,0.104728,-0.008876,-0.056841,0.229139,0.127199,-0.007909,-0.000406,-0.022714,0.110767,0.018240,0.051083,0.417098,-0.036104,0.012223,-0.298821,0.079250,0.118803,0.223140
precipitacion_cesar,0.085191,1.000000,0.006276,-0.023416,0.036497,0.049240,0.361133,0.061164,-0.077214,0.142740,0.095596,0.059109,-0.065933,0.109118,0.043014,-0.155227,0.018447,0.002816,-0.111565,0.230750,-0.015286,0.185575,-0.033670,0.204421,0.014740,-0.048245,0.042398,-0.068896,-0.017470,0.014878,0.168497
precipitacion_putumayo,0.299014,0.006276,1.000000,-0.054078,0.012250,-0.051010,0.070477,0.056463,0.032072,-0.079292,0.055070,0.221635,0.039481,0.033521,-0.019715,-0.145711,0.155397,0.062644,0.035349,0.009300,-0.000844,-0.035423,0.013209,-0.153138,0.161010,0.134876,-0.043160,-0.176497,0.137690,0.111173,0.112979
temp_AMAZONAS,-0.023762,-0.023416,-0.054078,1.000000,0.133536,0.108251,-0.085075,0.038514,0.089333,0.007794,-0.060151,-0.153805,0.408089,-0.056657,0.062327,0.167044,0.109058,-0.007973,-0.136227,-0.011762,-0.232514,-0.084397,0.337004,-0.195409,-0.154113,0.269572,0.058454,0.354338,0.024241,0.038957,-0.110378
precipitacion_atlantico,0.019529,0.036497,0.012250,0.133536,1.000000,0.114213,0.051031,0.017710,-0.181007,0.119168,0.116575,0.057876,-0.052018,0.013352,0.230301,0.041105,0.052842,0.137871,-0.007299,0.053496,-0.075014,0.204079,0.117832,0.115467,0.060673,0.085676,-0.045853,0.070169,-0.050822,-0.019426,0.119205
precipitacion_sucre,0.066551,0.049240,-0.051010,0.108251,0.114213,1.000000,0.085436,0.190068,-0.125405,0.340527,0.065659,0.191356,-0.069105,0.075289,0.034341,0.162858,0.094089,0.161990,-0.328135,0.224334,0.008736,0.240688,0.107107,0.233963,0.095510,0.244410,0.118262,-0.062561,0.133493,0.013979,0.192362
precipitacion_risaralda,0.092406,0.361133,0.070477,-0.085075,0.051031,0.085436,1.000000,0.096886,-0.134231,0.130465,-0.007864,0.238492,-0.164941,0.235644,0.051087,-0.001091,0.078456,0.071756,-0.026154,0.143772,0.028251,0.214872,0.051390,0.183884,0.156547,0.018057,0.033057,-0.131445,0.070233,0.055010,0.152901
precipitacion_arauca,0.209292,0.061164,0.056463,0.038514,0.017710,0.190068,0.096886,1.000000,-0.109112,0.247388,0.107836,-0.007031,0.017726,0.045715,0.039089,0.084866,0.208583,0.142209,-0.260864,-0.047415,-0.066088,0.109131,0.185561,0.034924,0.247661,-0.017620,0.380051,-0.208100,0.130980,0.287361,0.117684
temp_CHOCO,-0.103962,-0.077214,0.032072,0.089333,-0.181007,-0.125405,-0.134231,-0.109112,1.000000,-0.152417,-0.020749,-0.069635,0.151223,-0.045204,-0.128843,0.350225,0.025559,-0.151070,0.193751,-0.029740,0.127459,-0.102299,-0.067539,-0.356461,-0.119032,0.451856,0.176241,0.272976,0.119331,-0.038585,-0.020139
precipitacion_bolivar,0.063806,0.142740,-0.079292,0.007794,0.119168,0.340527,0.130465,0.247388,-0.152417,1.000000,0.073121,0.090713,-0.110049,0.074195,0.138250,0.043375,0.091918,0.120488,-0.166742,0.041052,-0.033369,0.235722,0.003807,0.191135,0.089413,-0.004278,0.158988,-0.077080,0.145462,0.047074,0.167940


In [11]:
df_without_correlation["Date"] = df_merged["Date"]
df_without_correlation["energy_price"] = df_merged["energy_price"]
df_without_correlation.to_csv(
    "processed_tables/merged_no_damns_standardized.csv", index=False
)

/var/folders/nz/j6px70h134sbbj9vwy0gpzm40000gn/T/ipykernel_45208/3190552356.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_correlation["Date"] = df_merged["Date"]
/var/folders/nz/j6px70h134sbbj9vwy0gpzm40000gn/T/ipykernel_45208/3190552356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_correlation["energy_price"] = df_merged["energy_price"]
